In [2]:
import os
import requests

from db_connection import get_client

In [3]:
client = get_client()
hf_token = os.getenv('HF_TOKEN')
embedding_url = os.getenv('EMBEDDING_URL')
collection = client.sample_mflix.movies

Connection to MongoDB is succesful!


In [11]:
def generate_embedding(text):
    response = requests.post(
        embedding_url,
        headers={"Authorization": f"Bearer {hf_token}"},
        json={"inputs": text}
    )
    return response.json()

# Test the function with the text "MongoDB is awesome!"
embedding = generate_embedding("MongoDB is awesome!")
print(embedding[:3])

[-0.014143984764814377, -0.020810438320040703, -0.062023792415857315]


In [21]:
# Create vector embeddings for 50 docs in dataset, that have the field 'plot'.
# Store the result as 'plot_embedding_f' in db

for doc in collection.find({'plot':{"$exists": True}}).limit(50):
	doc['plot_embedding_hf'] = generate_embedding(doc['plot'])
	collection.replace_one({'_id': doc['_id']}, doc)

In [22]:
query = "imaginary characters from outer space at war"

results = collection.aggregate([
  {"$vectorSearch": {
    "queryVector": generate_embedding(query),
    "path": "plot_embedding_hf",
    "numCandidates": 100,
    "limit": 4,
    "index": "PlotSemanticSearch",
      }}
]);

for document in results:
    print(f'Movie Name: {document["title"]},\nMovie Plot: {document["plot"]}\n')

Movie Name: Four Sons,
Movie Plot: A family saga in which three of a Bavarian widow's sons go to war for Germany and the fourth goes to America, Germany's eventual opponent.

Movie Name: The Strong Man,
Movie Plot: A meek Belgian soldier (Harry Langdon) fighting in World War I receives penpal letters and a photo from "Mary Brown", an American girl he has never met. He becomes infatuated with her by ...

Movie Name: The Four Horsemen of the Apocalypse,
Movie Plot: An extended family split up in France and Germany find themselves on opposing sides of the battlefield during World War I.

Movie Name: In the Land of the Head Hunters,
Movie Plot: Original advertising for the film describes it as a drama of primitive life on the shores of the North Pacific...

